**Imports**

In [1]:
import yfinance as yf
from datetime import datetime
import pandas as pd
from typing import Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import math
import QuantLib as ql

**Define price fetcher**

In [2]:
def get_spot_price(ticker):
    """
    Fetches the current spot price for a stock ticker. Falls back to the most recent
    close if a live price is not available.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').

    Returns
    -------
    float or None
        Spot price (live if available, else last close). Returns None if unavailable.
    """

    try:
        tk = yf.Ticker(ticker)
        
        # Attempt to fetch live price
        live_price = tk.fast_info.get("last_price", None)
        if live_price and live_price > 0:
            return live_price

        # Fallback: most recent close
        hist = tk.history(period="1d")
        if not hist.empty:
            fallback_price = hist["Close"].iloc[-1]
            print(f"[{ticker}] Live price unavailable — using last close: {fallback_price:.2f}")
            return fallback_price

        print(f"[{ticker}] No live or historical data available.")
        return None

    except Exception as e:
        print(f"[{ticker}] Spot price fetch failed: {e}")
        return None

**Define Option Chain Fetcher**

In [3]:
def get_option_chains_all(ticker: str,
                                  max_workers: int = 8) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Fetches option chains (calls and puts) for every available expiry of a given ticker,
    performing API requests in parallel to reduce total fetch time.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').
    max_workers : int, optional
        Maximum number of threads to use for concurrent fetching (default is 8).

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        - calls_df: DataFrame containing all calls across expiries, with added columns:
            * 'option_type' = 'call'
            * 'expiration'  = expiry date string 'YYYY-MM-DD'
            * 'TTM'         = time to maturity in years
        - puts_df: DataFrame containing all puts with the same added columns.
    """
    stock = yf.Ticker(ticker)
    expiries = stock.options  # list of expiry date strings
    today = datetime.now().date()

    calls_accum = []
    puts_accum  = []

    def fetch_chain(expiry: str):
        """Fetch calls/puts for a single expiry and return (expiry, calls_df, puts_df)."""
        try:
            chain = stock.option_chain(expiry)
            calls = chain.calls.copy()
            puts  = chain.puts.copy()
        except Exception as e:
            # Return None on error so we can skip later
            return expiry, None, None

        # Tag each row with type and expiration
        calls['option_type']  = 'call'
        puts ['option_type']  = 'put'
        calls['expiration']   = expiry
        puts ['expiration']   = expiry

        # Compute time-to-maturity once
        exp_date = datetime.strptime(expiry, "%Y-%m-%d").date()
        ttm = max((exp_date - today).days / 365.0, 0.0)
        calls['TTM'] = ttm
        puts ['TTM'] = ttm

        return expiry, calls, puts

    # Fetch in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_chain, exp) for exp in expiries]
        for future in as_completed(futures):
            expiry, calls_df, puts_df = future.result()
            if calls_df is not None and not calls_df.empty:
                calls_accum.append(calls_df)
            if puts_df  is not None and not puts_df.empty:
                puts_accum.append(puts_df)

    # Concatenate results
    all_calls = pd.concat(calls_accum, ignore_index=True) if calls_accum else pd.DataFrame()
    all_puts  = pd.concat(puts_accum,  ignore_index=True) if puts_accum  else pd.DataFrame()

    # Fetch dividend yield for the company
    dividendYield = stock.info.get("dividendYield")/100 # percentages on decimal basis
    all_calls["dividendYield"] = dividendYield
    all_puts["dividendYield"] = dividendYield

    all_calls["ticker"] = ticker
    all_puts["ticker"] = ticker

    spot_price = get_spot_price(ticker)
    all_calls["spot_price"] = spot_price
    all_puts["spot_price"] = spot_price

    return all_calls, all_puts


Get option chain and dividend yield

In [4]:
calls, puts = get_option_chains_all("JPM")

[JPM] Live price unavailable — using last close: 314.21


In [5]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251114C00160000,2025-10-30 13:50:42+00:00,160.0,150.00,152.30,156.15,0.000000,0.000000,2.0,4,1.484378,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
1,JPM251114C00205000,2025-10-22 16:30:41+00:00,205.0,87.50,107.40,110.85,0.000000,0.000000,NaN,1,1.820802,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
2,JPM251114C00225000,2025-10-22 16:30:50+00:00,225.0,67.05,87.40,90.95,0.000000,0.000000,NaN,1,1.506350,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
3,JPM251114C00240000,2025-10-22 16:30:45+00:00,240.0,52.75,72.40,75.65,0.000000,0.000000,NaN,2,1.212895,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
4,JPM251114C00245000,2025-10-22 16:30:51+00:00,245.0,47.40,67.45,70.00,0.000000,0.000000,NaN,2,1.000982,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,0.251671,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
880,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,0.248192,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
881,JPM271217C00450000,2025-10-28 13:38:06+00:00,450.0,8.33,10.45,11.50,0.000000,0.000000,NaN,1,0.249809,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
882,JPM271217C00460000,2025-10-09 16:57:09+00:00,460.0,7.50,7.00,10.90,0.000000,0.000000,NaN,2,0.254173,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991


In [6]:
puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.0,0.00000,5.0,6,1.812501,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
1,JPM251114P00170000,2025-10-22 19:01:42+00:00,170.0,0.04,0.00,2.13,0.0,0.00000,NaN,2,2.286625,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
2,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.1,-71.42857,1.0,1,1.425784,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
3,JPM251114P00190000,2025-10-21 18:52:08+00:00,190.0,0.03,0.00,2.12,0.0,0.00000,NaN,4,1.919434,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
4,JPM251114P00195000,2025-10-06 17:31:40+00:00,195.0,0.24,0.00,2.12,0.0,0.00000,NaN,1,1.833985,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,JPM271217P00370000,2025-10-21 18:48:09+00:00,370.0,80.30,69.00,74.00,0.0,0.00000,8.0,31,0.203591,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991
779,JPM271217P00390000,2025-08-07 18:50:02+00:00,390.0,103.85,96.55,100.00,0.0,0.00000,NaN,2,0.266716,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991
780,JPM271217P00400000,2025-09-05 17:33:06+00:00,400.0,107.07,94.00,97.80,0.0,0.00000,1.0,7,0.202889,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991
781,JPM271217P00410000,2025-09-29 16:41:08+00:00,410.0,99.90,99.00,103.00,0.0,0.00000,3.0,2,0.178002,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991


In [7]:
calls.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,884.000000,884.000000,884.000000,884.000000,884.000000,884.000000,837.000000,884.000000,884.000000,884.000000,884.0000,884.000000
mean,279.736991,59.886765,61.534706,63.706968,0.009514,1.305206,65.540024,436.348416,0.433174,0.635917,0.0191,314.209991
std,94.121347,59.276993,61.384402,62.465660,1.283287,34.770268,362.538300,993.660580,0.387665,0.652016,0.0000,0.000000
min,65.000000,0.010000,0.000000,0.000000,-5.450005,-98.461540,1.000000,0.000000,0.000010,0.000000,0.0191,314.209991
25%,200.000000,7.097500,6.487500,7.737500,0.000000,0.000000,2.000000,12.000000,0.259918,0.115068,0.0191,314.209991
50%,287.500000,38.625000,38.475000,40.750000,0.000000,0.000000,4.000000,72.000000,0.327292,0.364384,0.0191,314.209991
75%,350.000000,107.340000,111.487500,114.225000,0.000000,0.000000,20.000000,349.750000,0.482732,1.112329,0.0191,314.209991
max,470.000000,240.200000,242.750000,246.100000,13.110001,800.000240,6744.000000,8789.000000,6.646486,2.205479,0.0191,314.209991


Initial Dataset Clean

In [8]:
def initial_dataset_clean(dataset : pd.DataFrame):
    # Removing columns that will not be used for training or filtering
    #cols_to_drop = ["contractSymbol", "lastTradeDate", "change", "percentChange", "expiration", "inTheMoney"]
    #dataset.drop(columns=[c for c in cols_to_drop if c in dataset.columns], inplace=True)

    # Removing NA values for stale options
    dataset.dropna(axis=0,subset=["volume"], inplace=True)

    
    return dataset

In [9]:
calls_clean = initial_dataset_clean(calls)
calls_clean

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251114C00160000,2025-10-30 13:50:42+00:00,160.0,150.00,152.30,156.15,0.000000,0.000000,2.0,4,1.484378,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
7,JPM251114C00270000,2025-11-06 15:14:00+00:00,270.0,41.50,42.50,45.30,0.000000,0.000000,5.0,6,0.721194,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
8,JPM251114C00275000,2025-11-03 18:55:24+00:00,275.0,35.64,37.50,39.85,0.000000,0.000000,10.0,21,0.576909,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
9,JPM251114C00280000,2025-11-03 18:55:24+00:00,280.0,30.71,32.55,35.25,0.000000,0.000000,10.0,25,0.577397,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
11,JPM251114C00285000,2025-10-30 13:50:20+00:00,285.0,24.90,27.60,30.10,0.000000,0.000000,1.0,6,0.489019,True,REGULAR,USD,call,2025-11-14,0.019178,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,0.255188,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
878,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,0.253212,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
879,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,0.251671,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
880,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,0.248192,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991


In [10]:
calls_clean.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,8.370000e+02,837.000000
mean,280.400239,59.668220,61.124504,63.278292,0.010048,1.378497,65.540024,460.659498,0.428916,0.644006,1.910000e-02,314.209991
std,93.288559,59.706082,61.695467,62.784608,1.318865,35.732882,362.538300,1015.745730,0.388136,0.653501,3.471521e-18,0.000000
min,65.000000,0.010000,0.000000,0.000000,-5.450005,-98.461540,1.000000,0.000000,0.000010,0.000000,1.910000e-02,314.209991
25%,205.000000,7.000000,6.500000,7.800000,0.000000,0.000000,2.000000,18.000000,0.259926,0.115068,1.910000e-02,314.209991
50%,290.000000,37.330000,37.250000,39.700000,0.000000,0.000000,4.000000,82.000000,0.323218,0.364384,1.910000e-02,314.209991
75%,350.000000,107.010000,110.550000,113.300000,0.000000,0.000000,20.000000,384.000000,0.468999,1.112329,1.910000e-02,314.209991
max,470.000000,240.200000,242.750000,246.100000,13.110001,800.000240,6744.000000,8789.000000,6.646486,2.205479,1.910000e-02,314.209991


In [11]:
puts_clean = initial_dataset_clean(puts)
puts_clean

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.0,0.00000,5.0,6,1.812501,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
2,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.1,-71.42857,1.0,1,1.425784,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
5,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.0,0.00000,2.0,4,1.750001,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
6,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.0,0.00000,1.0,2,1.532717,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
7,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.0,0.00000,5.0,15,0.906251,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,JPM271217P00350000,2025-10-21 18:49:29+00:00,350.0,66.55,57.70,60.55,0.0,0.00000,20.0,39,0.209763,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991
777,JPM271217P00360000,2025-10-21 18:48:14+00:00,360.0,73.20,64.15,67.10,0.0,0.00000,8.0,21,0.206642,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991
778,JPM271217P00370000,2025-10-21 18:48:09+00:00,370.0,80.30,69.00,74.00,0.0,0.00000,8.0,31,0.203591,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991
780,JPM271217P00400000,2025-09-05 17:33:06+00:00,400.0,107.07,94.00,97.80,0.0,0.00000,1.0,7,0.202889,True,REGULAR,USD,put,2027-12-17,2.109589,0.0191,JPM,314.209991


**Join Calls and Puts**

In [12]:
df = pd.concat([puts_clean, calls_clean], ignore_index=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.000000,0.000000,5.0,6,1.812501,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
1,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.100000,-71.428570,1.0,1,1.425784,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
2,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.000000,0.000000,2.0,4,1.750001,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
3,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.000000,0.000000,1.0,2,1.532717,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
4,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.000000,0.000000,5.0,15,0.906251,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,0.255188,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
1571,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,0.253212,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
1572,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,0.251671,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991
1573,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,0.248192,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991


In [13]:
def optionType(OPTtype):
    # Puts are type 1
    if OPTtype == "put":
        return "put"
    else:
    # Calls are type 0
        return "call"

In [14]:
df["optionType"] = df["option_type"].apply(optionType)
#df.drop(columns="option_type", inplace=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.000000,0.000000,5.0,6,...,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put
1,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.100000,-71.428570,1.0,1,...,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put
2,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.000000,0.000000,2.0,4,...,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put
3,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.000000,0.000000,1.0,2,...,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put
4,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.000000,0.000000,5.0,15,...,False,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,...,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call
1571,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,...,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call
1572,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,...,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call
1573,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,...,False,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call


**Interest Rate Interpolation**

In [15]:
from interest.interest_rates import fetch_treasury_yield_curve_latest, interest_rate_interpolation
as_of, curve_series = fetch_treasury_yield_curve_latest()
df["r"] = interest_rate_interpolation(curve_series, df["TTM"])

In [16]:

df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.000000,0.000000,5.0,6,...,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331
1,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.100000,-71.428570,1.0,1,...,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331
2,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.000000,0.000000,2.0,4,...,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331
3,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.000000,0.000000,1.0,2,...,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331
4,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.000000,0.000000,5.0,15,...,REGULAR,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,...,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504
1571,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,...,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504
1572,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,...,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504
1573,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,...,REGULAR,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504


**Forward log-Moneyness**

We decided to use forward log-moneyness because its adjusted for r and q which are crucial in pricing the EEP

In [17]:

def forward_log_moneyness(S: float, r:float, q:float, T:float, K:float):
    """
    Calculates the log_moneyness in relation to the Forward Price of the stock

    Parameters
    ----------
    S : float
        Current spot_price of the stock.
    r : float
        Risk-free rate associated with that particular option maturity
    q : float
        Dividend-yield rate associated with that particular option maturity
    T : float
        Maturity of the option contract
    K : float
        Strike price of the option contract

    Returns
    -------
    Float
        - Forward log moneyness 
    """
    forward_price = S * np.exp((r - q)*T)
    
    return np.log(K/forward_price)

In [18]:
df["forward_log_moneyness"] = forward_log_moneyness(
    S=df["spot_price"].to_numpy(),
    r=df["r"].to_numpy(),
    q=df["dividendYield"].to_numpy(),
    T=df["TTM"].to_numpy(),
    K=df["strike"].to_numpy(),
)

df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.000000,0.000000,5.0,6,...,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.675295
1,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.100000,-71.428570,1.0,1,...,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.557512
2,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.000000,0.000000,2.0,4,...,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.452151
3,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.000000,0.000000,1.0,2,...,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.403361
4,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.000000,0.000000,5.0,15,...,USD,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.379831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,...,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.231491
1571,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,...,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.255588
1572,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,...,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.279119
1573,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,...,USD,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.302108


Calculate Mid Price

In [19]:
def mid_price(bid: float, ask: float):
    """
    Calcualtes the mid-price of the option based on bid and ask prices

    Parameters
    ----------
    bid : float
        Current bid price of the option
    ask : float
        Current ask price of the option

    Returns
    -------
    Float
        - Mid price
    """

    return ((bid+ask)/2)
    

In [20]:
df["midPrice"] = mid_price(df["bid"], df["ask"])
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness,midPrice
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.000000,0.000000,5.0,6,...,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.675295,0.115
1,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.100000,-71.428570,1.0,1,...,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.557512,0.065
2,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.000000,0.000000,2.0,4,...,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.452151,1.060
3,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.000000,0.000000,1.0,2,...,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.403361,0.880
4,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.000000,0.000000,5.0,15,...,put,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.379831,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,...,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.231491,17.275
1571,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,...,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.255588,15.825
1572,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,...,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.279119,13.750
1573,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,...,call,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.302108,11.825


In [21]:
from heston.heston import calibrate_and_price_heston_european

V_eu = calibrate_and_price_heston_european(df, group_cols=("ticker", ))
df = df.join(V_eu)
df

[deAm] === group start === :: group=('JPM',) | rows=1575
[deAm] sigma* :: tree=jr | sigma_star=1.9417908934993187
[deAm] eu price :: tree=jr | p_eu=0.11575397690562907
[deAm] sigma* :: tree=jr | sigma_star=1.5278603982629049
[deAm] eu price :: tree=jr | p_eu=0.06552750699011953
[deAm] sigma* :: tree=jr | sigma_star=1.873257371715758
[deAm] eu price :: tree=jr | p_eu=1.062933755909612
[deAm] sigma* :: tree=jr | sigma_star=1.6399957625074617
[deAm] eu price :: tree=jr | p_eu=0.8796088158972276
[deAm] sigma* :: tree=jr | sigma_star=0.9740250925434901
[deAm] eu price :: tree=jr | p_eu=0.025229794212848523
[deAm] sigma* :: tree=jr | sigma_star=1.0925347861026808
[deAm] eu price :: tree=jr | p_eu=0.1905415319445549
[deAm] sigma* :: tree=jr | sigma_star=1.1764253766154213
[deAm] eu price :: tree=jr | p_eu=0.6534457253865147
[deAm] sigma* :: tree=jr | sigma_star=0.7656440321772998
[deAm] eu price :: tree=jr | p_eu=0.050652949490163135
[deAm] sigma* :: tree=jr | sigma_star=0.8860846805843237
[d

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness,midPrice,V_EU_Heston
0,JPM251114P00160000,2025-10-27 13:46:23+00:00,160.0,0.01,0.00,0.23,0.000000,0.000000,5.0,6,...,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.675295,0.115,1.241352e-10
1,JPM251114P00180000,2025-11-07 18:01:01+00:00,180.0,0.04,0.00,0.13,-0.100000,-71.428570,1.0,1,...,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.557512,0.065,9.148236e-09
2,JPM251114P00200000,2025-10-31 14:42:36+00:00,200.0,0.05,0.00,2.12,0.000000,0.000000,2.0,4,...,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.452151,1.060,4.159050e-07
3,JPM251114P00210000,2025-10-14 14:27:52+00:00,210.0,1.12,0.00,1.76,0.000000,0.000000,1.0,2,...,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.403361,0.880,2.408185e-06
4,JPM251114P00215000,2025-10-31 14:52:31+00:00,215.0,0.08,0.00,0.05,0.000000,0.000000,5.0,15,...,2025-11-14,0.019178,0.0191,JPM,314.209991,put,0.040331,-0.379831,0.025,5.600794e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM271217C00410000,2025-11-07 18:46:59+00:00,410.0,17.10,16.25,18.30,1.600000,10.322582,10.0,120,...,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.231491,17.275,2.073435e+01
1571,JPM271217C00420000,2025-10-29 18:58:44+00:00,420.0,12.40,15.40,16.25,0.000000,0.000000,1.0,40,...,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.255588,15.825,1.861589e+01
1572,JPM271217C00430000,2025-10-29 17:11:50+00:00,430.0,10.75,13.05,14.45,0.000000,0.000000,2.0,8,...,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.279119,13.750,1.670044e+01
1573,JPM271217C00440000,2025-11-07 19:50:17+00:00,440.0,12.15,11.10,12.55,2.259999,22.851357,10.0,125,...,2027-12-17,2.109589,0.0191,JPM,314.209991,call,0.035504,0.302108,11.825,1.497301e+01


In [22]:
df.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price,r,forward_log_moneyness,midPrice,V_EU_Heston
count,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1.575000e+03,1575.000000,1575.000000,1575.000000,1575.000000,1.575000e+03
mean,262.500000,37.428844,37.450444,39.068203,-0.022857,9.457881,56.989206,488.363175,0.469281,0.627869,1.910000e-02,314.209991,0.038123,-0.255126,38.259324,3.900012e+01
std,87.490288,51.703350,53.208820,54.254441,1.104338,182.361424,290.838084,969.676568,0.480324,0.654474,3.470549e-18,0.000000,0.001797,0.377693,53.730146,5.578327e+01
min,65.000000,0.010000,0.000000,0.000000,-11.660000,-98.461540,1.000000,0.000000,0.000010,0.000000,1.910000e-02,314.209991,0.035504,-1.578080,0.000000,-5.950684e-12
25%,190.000000,1.285000,0.920000,1.965000,0.000000,0.000000,2.000000,20.000000,0.259910,0.115068,1.910000e-02,314.209991,0.036360,-0.509323,1.172500,1.212767e+00
50%,270.000000,10.950000,10.450000,11.550000,0.000000,0.000000,5.000000,102.000000,0.329841,0.364384,1.910000e-02,314.209991,0.038055,-0.159748,10.775000,1.072229e+01
75%,321.250000,54.325000,52.700000,54.750000,0.000000,0.000000,22.000000,488.000000,0.488958,0.987671,1.910000e-02,314.209991,0.040003,0.017650,53.725000,5.429268e+01
max,470.000000,240.200000,242.750000,246.100000,13.110001,4800.004400,6744.000000,8789.000000,6.646486,2.205479,1.910000e-02,314.209991,0.040400,0.387519,244.425000,2.488211e+02


In [23]:
df.columns

Index(['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask',
       'change', 'percentChange', 'volume', 'openInterest',
       'impliedVolatility', 'inTheMoney', 'contractSize', 'currency',
       'option_type', 'expiration', 'TTM', 'dividendYield', 'ticker',
       'spot_price', 'optionType', 'r', 'forward_log_moneyness', 'midPrice',
       'V_EU_Heston'],
      dtype='object')